In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import tensorflow as tf


import warnings
warnings.filterwarnings("ignore")

from keras.layers import Input, Dense, Reshape, Flatten, Dropout, BatchNormalization, Embedding,Conv1D
from tensorflow.keras.layers import LeakyReLU
from keras.layers import concatenate
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
import scipy.stats
import tarfile
from pandas.errors import EmptyDataError
import time
import os

In [ ]:
df=pd.read_csv('Combined_IoT_profiling_dataset_2022_train_set.csv')

df

,Unnamed: 0,L4_tcp,L4_udp,L7_http,L7_https,port_class_src,port_class_dst,pck_size,ip_dst_new,ethernet_frame_size,...,skew_e,kurt_e,var_e,q3_e,q1_e,iqr_e,epoch_timestamp,inter_arrival_time,time_since_previously_displayed_frame,label
0,4,0,1,0,0,2,2,36,3758096635,70,...,0.108799,-1.815521,2405.918367,130.00,28.00,102.00,1.634050e+09,0.000000,4.515739,Audio_nestmini
1,2039,1,0,0,0,3,2,130,3232270754,164,...,1.531536,0.524287,53285.421488,130.00,35.00,95.00,1.634051e+09,4.961714,10.032669,Audio_nestmini
2,1224,0,1,0,0,2,1,318,2899944708,352,...,3.746280,12.568006,89396.127424,33.00,33.00,0.00,1.634055e+09,0.000000,1.239046,Audio_nestmini
3,1080,0,1,0,0,1,2,33,3232270754,67,...,0.917451,-0.946318,292253.765432,825.25,41.25,784.00,1.634055e+09,0.000085,6.532723,Audio_nestmini
4,595,0,1,0,0,2,2,47,3758096635,81,...,2.962775,7.228718,26089.254438,68.00,40.00,28.00,1.634054e+09,1.286575,10.282488,Audio_nestmini
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184830,206843,0,1,0,0,2,3,60,3232270721,94,...,-0.825583,-1.165924,192593.015432,1096.00,408.00,688.00,1.634046e+09,0.017099,27.400710,Camera_simcam
184831,218333,0,1,0,0,2,3,60,3232270721,94,...,-0.101472,-1.801175,215640.062500,1096.00,60.00,1036.00,1.634046e+09,0.000056,29.139172,Camera_simcam
184832,200791,0,1,0,0,3,2,1096,2211110130,1130,...,0.000000,-3.000000,0.000000,1096.00,1096.00,0.00,1.634046e+09,0.000000,11.267276,Camera_simcam
184833,224393,0,1,0,0,3,2,1116,2211110130,1150,...,2.846050,6.100000,33.057851,1096.00,1096.00,0.00,1.634046e+09,0.000000,12.436347,Camera_simcam


In [ ]:

mask =df['label'].str.contains('Camera')
df.loc[mask, 'label'] = 'Camera'
mask =df['label'].str.contains('Audio')
df.loc[mask, 'label'] = 'Audio'
mask = df['label'].str.contains('Home Automation')
df.loc[mask, 'label'] = 'Home Automation'

In [ ]:
df.label.value_counts()

label
Camera             153937
Home Automation     15864
Audio               15034
Name: count, dtype: int64

In [ ]:
os.getcwd()
os.chdir('../../../../../../')
os.getcwd()

'/'

In [ ]:
camera_df=df[df['label']=='Camera']
camera_df.label.value_counts()
camera_df=camera_df.drop(['label'],axis=1)
camera_df.to_csv('IoT_camera.csv')

In [ ]:
camera_df

,Unnamed: 0,L4_tcp,L4_udp,L7_http,L7_https,port_class_src,port_class_dst,pck_size,ip_dst_new,ethernet_frame_size,...,average,skew_e,kurt_e,var_e,q3_e,q1_e,iqr_e,epoch_timestamp,inter_arrival_time,time_since_previously_displayed_frame
30898,48170,0,1,0,0,3,2,1388,2385310024,1422,...,1185.894737,-1.876388,1.520833,217848.199446,1388.0,1388.00,0.00,1.633542e+09,0.000625,10.335814
30899,48819,0,1,0,0,3,2,1388,2385310024,1422,...,1228.000000,-2.267787,3.142857,179200.000000,1388.0,1388.00,0.00,1.633542e+09,0.005834,14.333014
30900,45014,0,1,0,0,3,2,1388,2385310024,1422,...,1174.666667,-1.788854,1.200000,227555.555556,1388.0,1388.00,0.00,1.633542e+09,0.000000,13.044197
30901,46750,0,1,0,0,3,2,1388,2385310024,1422,...,1320.631579,-4.006938,14.055556,81693.074792,1388.0,1388.00,0.00,1.633542e+09,0.000883,2.659452
30902,40794,0,1,0,0,3,2,1388,2385310024,1422,...,748.000000,0.000000,-2.000000,409600.000000,1388.0,108.00,1280.00,1.633542e+09,0.000603,8.510484
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184830,206843,0,1,0,0,2,3,60,3232270721,94,...,780.611111,-0.825583,-1.165924,192593.015432,1096.0,408.00,688.00,1.634046e+09,0.017099,27.400710
184831,218333,0,1,0,0,2,3,60,3232270721,94,...,618.750000,-0.101472,-1.801175,215640.062500,1096.0,60.00,1036.00,1.634046e+09,0.000056,29.139172
184832,200791,0,1,0,0,3,2,1096,2211110130,1130,...,1096.000000,0.000000,-3.000000,0.000000,1096.0,1096.00,0.00,1.634046e+09,0.000000,11.267276
184833,224393,0,1,0,0,3,2,1116,2211110130,1150,...,1097.818182,2.846050,6.100000,33.057851,1096.0,1096.00,0.00,1.634046e+09,0.000000,12.436347


In [ ]:
audio_df=df[df['label']=='Audio']
print(audio_df.label.value_counts())
audio_df=audio_df.drop(['label'],axis=1)
audio_df.to_csv('IoT_audio.csv')

label
Audio    15034
Name: count, dtype: int64


In [ ]:
audio_df

,L4_tcp,L4_udp,L7_http,L7_https,port_class_src,port_class_dst,pck_size,ip_dst_new,ethernet_frame_size,ttl,...,average,skew_e,kurt_e,var_e,q3_e,q1_e,iqr_e,epoch_timestamp,inter_arrival_time,time_since_previously_displayed_frame
210983,1,0,0,1,2,1,302,880264544,336,64,...,302.000000,0.000000,-3.000000,0.000000,302.0,302.0,0.0,1.633611e+09,0.000000,0.000000
210984,1,0,0,1,2,1,1480,880264544,1514,64,...,891.000000,0.000000,-2.000000,346921.000000,1185.5,596.5,589.0,1.633611e+09,0.007527,0.007527
210985,1,0,0,1,2,1,1480,880264544,1514,64,...,1087.333333,-0.707107,-1.500000,308374.222222,1480.0,891.0,589.0,1.633611e+09,0.000000,0.007527
210986,1,0,0,1,2,1,1480,880264544,1514,64,...,1185.500000,-1.154701,-0.666667,260190.750000,1480.0,1185.5,294.5,1.633611e+09,0.000770,0.008297
210987,1,0,0,1,2,1,1480,880264544,1514,64,...,1244.400000,-1.500000,0.250000,222029.440000,1480.0,1480.0,0.0,1.633611e+09,0.000000,0.008297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229752,0,1,0,0,2,2,36,3758096635,70,255,...,64.375000,0.696762,-1.383412,2034.859375,130.0,32.0,98.0,1.634050e+09,0.000000,15.277532
229753,0,1,0,0,2,2,36,3758096635,70,255,...,62.705882,0.791517,-1.238852,1959.737024,130.0,32.0,98.0,1.634050e+09,0.000000,15.277532
229754,1,0,0,0,3,2,130,3232270754,164,128,...,66.444444,0.610920,-1.514330,2088.469136,130.0,32.0,98.0,1.634050e+09,0.096075,15.373607
229755,1,0,0,0,2,3,130,3232270593,164,64,...,69.789474,0.460170,-1.690172,2179.955679,130.0,32.0,98.0,1.634050e+09,0.014977,15.388584


In [ ]:
home_automation_df=df[df['label']=='Home Automation']
print(home_automation_df.label.value_counts())
home_automation_df=home_automation_df.drop(['label'],axis=1)
home_automation_df.to_csv('IoT_home_automation.csv')

label
Home Automation    15864
Name: count, dtype: int64


In [ ]:
home_automation_df

,L4_tcp,L4_udp,L7_http,L7_https,port_class_src,port_class_dst,pck_size,ip_dst_new,ethernet_frame_size,ttl,...,average,skew_e,kurt_e,var_e,q3_e,q1_e,iqr_e,epoch_timestamp,inter_arrival_time,time_since_previously_displayed_frame
0,1,0,0,0,2,2,233,3232270836,267,224,...,197.666667,0.707107,-1.500000,624.222222,206.50,180.00,26.50,1.633458e+09,0.000000,0.000000
1,1,0,0,0,2,2,20,597822746,54,255,...,153.250000,-0.891663,-0.813579,6386.687500,193.25,140.00,53.25,1.633458e+09,0.214945,0.214945
2,1,0,0,0,2,2,201,597822746,235,255,...,162.800000,-1.234550,-0.047248,5474.160000,201.00,180.00,21.00,1.633458e+09,0.139723,0.354668
3,1,0,0,0,2,2,89,3232270836,123,224,...,150.500000,-0.736303,-0.886431,5318.250000,195.75,111.75,84.00,1.633458e+09,0.180066,0.534734
4,1,0,0,0,2,2,20,597822746,54,255,...,131.857143,-0.340153,-1.528323,6643.836735,190.50,54.50,136.00,1.633458e+09,0.181868,0.716602
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19803,1,0,0,0,2,3,89,3232270655,123,223,...,89.000000,0.000000,-3.000000,0.000000,89.00,89.00,0.00,1.633693e+09,0.087663,5.764593
19804,1,0,0,0,3,2,20,875292917,54,255,...,54.500000,0.000000,-2.000000,1190.250000,71.75,37.25,34.50,1.633693e+09,0.117175,5.881768
19805,0,1,0,0,3,2,196,4294967295,230,255,...,101.666667,0.257049,-1.500000,5242.888889,142.50,54.50,88.00,1.633693e+09,4.113669,9.995437
19806,0,0,0,0,0,0,0,0,60,0,...,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,1.633693e+09,2.317506,12.312943
